In [ ]:
import sympy as sp 
import numpy as np
from sympy.physics.quantum import TensorProduct as TP

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import Normalize

import cayley_schreier as cs

In [ ]:
params = {
    "N_pbc": 401, #number of k-points in the Brillouin zone
    "num_bands": 4, #number of bands
    "t1": 1.0, #hopping parameter model a and b
    "t2": 1.0, #hopping parameter model a and b
}

In [ ]:
#define symbols
t1, t2 = sp.symbols('t1 t2', real=True)
k = sp.symbols('k', real=True)
symbols = [k, t1, t2]

#create the Bloch Hamiltonians for models a and b
Hk_a = cs.create_triangular_hamiltonian(symbols, model = 'a')

In [ ]:
#substitute hopping parameters
Hk_a_subs = Hk_a.subs({t1:params["t1"], t2:params["t2"]})
Hk_a_f = sp.lambdify(k, Hk_a_subs, modules='numpy')

Ks = np.linspace(0, 2*np.pi, params["N_pbc"], endpoint=False)+1e-4

#pbc spectrum for Hk_a
eigenvals_a, eigenvecs_a = cs.spectrum(Hk_a_f, [Ks], params["num_bands"])
#make the bands continuous
eigenvecs_a, eigenvals_a = cs.bands.continuous_bands_1d(eigenvecs_a, eigenvals_a)

In [ ]:
Gy = cs.symmetry.create_Gy_unitary(ksym = k)
Gy_func = sp.lambdify(k, Gy, modules='numpy')

#define expectation values of Gy in the eigenstates of Hk_a
gy_exp = np.zeros((params["N_pbc"], params["num_bands"]), dtype = np.complex128)
for i in range(params["N_pbc"]):
    for n in range(params["num_bands"]):
        gy_exp[i,n] = (eigenvecs_a[i,:,n].conj().T @ Gy_func(Ks[i]) @ eigenvecs_a[i,:,n])

In [ ]:
x = Ks
y1 = eigenvals_a[:,0]
y2 = eigenvals_a[:,1]
y3 = eigenvals_a[:,2]
y4 = eigenvals_a[:,3]

# Example "phase values" (0 to 2π)
phase1 = np.angle(gy_exp[:,0]) 
phase2 = np.angle(gy_exp[:,1]) 
phase3 = np.angle(gy_exp[:,2])
phase4 = np.angle(gy_exp[:,3])

# Convert (x,y) into line segments
points1 = np.array([x, y1]).T.reshape(-1, 1, 2)
segments1 = np.concatenate([points1[:-1], points1[1:]], axis=1)

points2 = np.array([x, y2]).T.reshape(-1, 1, 2)
segments2 = np.concatenate([points2[:-1], points2[1:]], axis=1)

points3 = np.array([x, y3]).T.reshape(-1, 1, 2)
segments3 = np.concatenate([points3[:-1], points3[1:]], axis=1)

points4 = np.array([x, y4]).T.reshape(-1, 1, 2)
segments4 = np.concatenate([points4[:-1], points4[1:]], axis=1)

# Normalize
norm = Normalize(vmin=-np.pi, vmax=np.pi)

cmap = plt.cm.hsv

# Create LineCollection
lc1 = LineCollection(segments1, cmap=cmap, norm=norm)
lc1.set_array(phase1)
lc1.set_linewidth(2)

lc2 = LineCollection(segments2, cmap=cmap, norm=norm)
lc2.set_array(phase2)
lc2.set_linewidth(2)

lc3 = LineCollection(segments3, cmap=cmap, norm=norm)
lc3.set_array(phase3)
lc3.set_linewidth(2)

lc4 = LineCollection(segments4, cmap=cmap, norm=norm)
lc4.set_array(phase4)
lc4.set_linewidth(2)

# Plot
fig, ax = plt.subplots(figsize = (4,3))
ax.add_collection(lc1)
ax.add_collection(lc2)
ax.add_collection(lc3)
ax.add_collection(lc4)
ax.autoscale()

# Colorbar
cbar = plt.colorbar(lc1, ax=ax)
cbar.set_ticks([-np.pi, 0, np.pi])
cbar.set_ticklabels([r"$-\pi$", r"$0$", r"$\pi$"])
cbar.set_label(r"arg $\mathrm{G_y} $", fontsize = 14, labelpad=-5)

ax.set_xlabel(r"$\mathsf{k}$", fontsize=14, labelpad=0)
ax.set_ylabel(r"$\mathsf{\epsilon}$", fontsize=16, labelpad=-5)
ax.set_xticks([0, np.pi, 2*np.pi],
               labels=[r"$\mathsf{0}$", r"$\mathsf{\pi}$", r"$\mathsf{2\pi}$"],
               fontsize=14)
ax.set_yticks([-3, 0, 3],
               [r"$\mathsf{-3}$", r"$\mathsf{0}$", r"$\mathsf{3}$"],
               fontsize=14)
#plt.savefig("../figures/methodsfig_a.svg", dpi=1000, bbox_inches='tight')
plt.show()